In [2]:
import pyspark
from pyspark.sql import SparkSession



In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

df = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [4]:
df.write.parquet('zones')

In [5]:
!head taxi+_zone_lookup.csv

In [7]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

--2022-02-21 05:36:43--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.74.68
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.74.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752335705 (717M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-01.csv’

fhvhv_tripdata_2021 100%[===================>] 717.48M  1.31MB/s    in 7m 30s  

2022-02-21 05:44:13 (1.60 MB/s) - ‘fhvhv_tripdata_2021-01.csv’ saved [752335705/752335705]



In [8]:
!wc -l fhvhv_tripdata_2021-01.csv

 11908469 fhvhv_tripdata_2021-01.csv


In [9]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [10]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [11]:
!head -n 2000 fhvhv_tripdata_2021-01.csv > head.csv

In [12]:
import pandas as pd

In [13]:
df_pandas = pd.read_csv('head.csv')

In [14]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [15]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [16]:
from pyspark.sql import types

In [17]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [18]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [19]:
df = df.repartition(10)

In [20]:
df.write.parquet('fhvhv/2021/01/')

22/02/21 05:58:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/21 05:58:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
22/02/21 05:58:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
22/02/21 05:58:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
22/02/21 05:58:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [23]:
df = spark.read.parquet('fhvhv/2021/01/')

In [22]:
!pwd

/Users/gabidoye/Documents/Data_Engineering_course/week1


In [24]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [25]:
from pyspark.sql import functions as F

In [26]:
df.show(n=10)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02875|2021-01-03 14:07:36|2021-01-03 14:14:11|          78|          60|   null|
|           HV0005|              B02510|2021-01-01 07:42:55|2021-01-01 07:46:57|          42|          41|   null|
|           HV0003|              B02880|2021-01-03 14:38:35|2021-01-03 14:50:39|          26|          26|   null|
|           HV0003|              B02764|2021-01-03 04:48:43|2021-01-03 04:53:47|         129|         138|   null|
|           HV0003|              B02867|2021-01-01 18:05:48|2021-01-01 18:23:20|         178|          85|   null|
|           HV0005|              B02510|2021-01-01 01:19:34|2021-01-01 01:28:06|

In [27]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [28]:
crazy_stuff('B02884')

's/b44'

In [29]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [30]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/b3b| 2021-01-03|  2021-01-03|          78|          60|
|  e/9ce| 2021-01-01|  2021-01-01|          42|          41|
|  a/b40| 2021-01-03|  2021-01-03|          26|          26|
|  e/acc| 2021-01-03|  2021-01-03|         129|         138|
|  e/b33| 2021-01-01|  2021-01-01|         178|          85|
|  e/9ce| 2021-01-01|  2021-01-01|          61|         225|
|  s/b3d| 2021-01-01|  2021-01-01|         258|         169|
|  e/a39| 2021-01-02|  2021-01-02|          77|          17|
|  e/b3c| 2021-01-03|  2021-01-03|         188|         231|
|  e/9ce| 2021-01-03|  2021-01-03|          48|         138|
|  e/b3f| 2021-01-01|  2021-01-01|         254|          32|
|  e/9ce| 2021-01-03|  2021-01-03|          74|         254|
|  a/b49| 2021-01-01|  2021-01-01|         179|          83|
|  e/b47| 2021-01-04|  2

In [32]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003') \
  .show(n=20)

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-03 14:07:36|2021-01-03 14:14:11|          78|          60|
|2021-01-03 14:38:35|2021-01-03 14:50:39|          26|          26|
|2021-01-03 04:48:43|2021-01-03 04:53:47|         129|         138|
|2021-01-01 18:05:48|2021-01-01 18:23:20|         178|          85|
|2021-01-01 18:45:51|2021-01-01 19:17:11|         258|         169|
|2021-01-02 18:52:48|2021-01-02 19:17:44|          77|          17|
|2021-01-03 11:52:31|2021-01-03 12:16:54|         188|         231|
|2021-01-01 00:30:59|2021-01-01 00:37:03|         254|          32|
|2021-01-01 01:17:49|2021-01-01 01:28:14|         179|          83|
|2021-01-04 01:07:33|2021-01-04 01:13:59|         130|         205|
|2021-01-04 11:44:30|2021-01-04 12:27:49|         174|          90|
|2021-01-01 09:51:19|2021-01-01 09:58:19|       

In [33]:
!head -n 10 head.csv